In [2]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import re, os

In [ ]:
driver = webdriver.Firefox();
driver.get("http://www.zerozero.pt/player_results.php?id=159904&competicao_id=3&epoca_id=147");
try:
    driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/button").click();
except:
    pass
driver.refresh();
#Python Calculate nth Root Function (Square, Cube, Fourth...)

# Eltiro de Ludantoj

### Retirar Equipas

In [ ]:
def get_teams():
    try:
        driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/button").click();
    except:
        pass
    
    #Site com equipas na tabela!
    driver.get('http://www.zerozero.pt/edition.php?simp=0&id_edicao=109369');
    #Refresh para retirar a publicidade!
    driver.refresh()
    
    #xpath para a tabela => ¡atencao jah deve ter mudado!
    table_ekipas="/html/body/div[7]/div[3]/div[6]/div[3]/div/div[3]/div[3]/table/tbody"
    trs=driver.find_elements_by_xpath(table_ekipas+"/tr") # => horizontal
    tds=driver.find_elements_by_xpath(table_ekipas+"/tr[1]/td") # => vertical
    print(len(trs), len(tds))
    
    #teamoj => nome de equipas, teamojn => link respectivo
    teamoj = []
    teamojn = []
    for i in range(1, len(trs)+1):
        current_team = driver.find_element_by_xpath((table_ekipas+"/tr[{}]/td[3]/a").format(str(i)))
        
        #link de cada equipa
        current_team_link = ((current_team).get_attribute('href'))
        equalSign = current_team_link.index('=')
        andSign = current_team_link.index('&')
        #extracao de ID unico de cada equipa
        #Por exemplo http://www.zerozero.pt/equipa.php?id=7&epoca_id=147 => fica ¡7!
        id_equipa = current_team_link[equalSign+1:andSign]
        
        teamoj.append(current_team) # Nome da equipa
        teamojn.append((current_team).get_attribute('href')) # Link da equipa => para colocar em ficheiro .txt
    for e in teamoj:
        if not os.path.exists('2017_2018'):
            os.makedirs('2017_2018/' + e.text)
        else:
            contrb = (str(e.text)+','+str((e).get_attribute('href')))
            print('2017_2018/' + e.text + '/equipaID.link')
            with open('2017_2018/' + e.text + '/equipaID.link', 'w+') as f:
                f.write(contrb)
    return teamoj,teamojn

### RETIRAR GIOCATORI

In [ ]:
def get_players(link_equipa):
    save : list = [] #['FC Porto','Benfica','Sporting','Braga','Rio Ave','Chaves','Marítimo','Boavista','V. Guimarães','Portimonense','Tondela','Belenenses']
    driver.get(link_equipa)
    id_equipa = driver.current_url
    print(id_equipa)
    nome_equipa = driver.find_element_by_class_name('name').text
    print(nome_equipa)
    if nome_equipa in save:
        return
    equalSign = id_equipa.index('=')
    andSign = id_equipa.index('&')
    id_equipa = id_equipa[equalSign+1:andSign]
    link = ('http://www.zerozero.pt/team_players.php?id={}&epoca_stats_id=147').format(id_equipa)
    driver.get(link)
    driver.refresh()
    time.sleep(0.5)
    table_ekipas = "/html/body/div[8]/div[3]/div[6]/div[3]/div/div/table[1]/tbody/"
    trs=driver.find_elements_by_xpath(table_ekipas+"/tr") # => horizontal
    tds=driver.find_elements_by_xpath(table_ekipas+"/tr[1]/td") # => vertical
    if len(trs) == 0:
        table_ekipas = "/html/body/div[7]/div[3]/div[6]/div[3]/div/div/table[1]/tbody/"
        trs=driver.find_elements_by_xpath(table_ekipas+"/tr") # => horizontal
        tds=driver.find_elements_by_xpath(table_ekipas+"/tr[1]/td") # => vertical
        
        
    print(len(tds),len(trs))
    for j in range(1, len(trs)+1):
        jogador = driver.find_element_by_xpath((table_ekipas+"/tr[{}]/td[3]/div/div[2]/a").format(str(j)))
        contrb = (jogador.text+','+(jogador.get_attribute('href'))+'\n')
        with open('2017_2018/'+nome_equipa+'/jogadores.jfo', 'a', encoding="utf-8") as f:
            f.write(contrb)

### Juegos por Jugador

In [ ]:
files = os.listdir('2017_2018')
for e in range(16, len(files)+1):
    print(files[e])#aqui
    path = '2017_2018/' + files[e] + '/jogadores.jfo'; #mudar aqui
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            jogador = line.split(',')
            if files[1] != 'FC Porto':
                driver.get(jogador[1])
                player_link = driver.current_url
                equalSign = player_link.index('=')
                id_jogador = player_link[equalSign+1:]
                link_jogos = (('http://www.zerozero.pt/player_results.php?id={}&competicao_id=3&epoca_id=147').format(id_jogador))
                driver.get(link_jogos)
                #/html/body/div[8]/div[3]/div[6]/div[3]/div/div/div/div[3]/div[2]/table/tbody
                table = '/html/body/div[6]/div[3]/div[6]/div[3]/div/div/div/div[3]/div[2]/table/tbody'
                trs = driver.find_elements_by_xpath(table+"/tr") # => horizontal
                tds = driver.find_elements_by_xpath(table+"/tr[1]/td") # => vertical
                for j in range(1, len(trs)+1):
                    try:
                        current_game = driver.find_element_by_xpath(('/html/body/div[8]/div[3]/div[6]/div[3]/div/div/div/div[3]/div[2]/table/tbody/tr[{}]/td[7]/a').format(j));
                        entrou_saiu = driver.find_element_by_xpath(('/html/body/div[8]/div[3]/div[6]/div[3]/div/div/div/div[3]/div[2]/table/tbody/tr[{}]/td[9]').format(j));
                        vitderr = driver.find_element_by_xpath(('/html/body/div[8]/div[3]/div[6]/div[3]/div/div/div/div[3]/div[2]/table/tbody/tr[{}]/td[1]/div').format(j));
                    except:
                        try:
                            current_game = driver.find_element_by_xpath(('/html/body/div[6]/div[3]/div[6]/div[3]/div/div/div/div[3]/div[2]/table/tbody/tr[{}]/td[7]/a').format(j));
                            entrou_saiu = driver.find_element_by_xpath(('/html/body/div[6]/div[3]/div[6]/div[3]/div/div/div/div[3]/div[2]/table/tbody/tr[{}]/td[9]').format(j));
                            vitderr = driver.find_element_by_xpath(('/html/body/div[6]/div[3]/div[6]/div[3]/div/div/div/div[3]/div[2]/table/tbody/tr[{}]/td[1]/div').format(j));
                        except:
                            pass
                    try:
                        current_game = driver.find_element_by_xpath(('/html/body/div[7]/div[3]/div[6]/div[3]/div/div/div/div[3]/div[2]/table/tbody/tr[{}]/td[7]/a').format(j));
                        entrou_saiu = driver.find_element_by_xpath(('/html/body/div[7]/div[3]/div[6]/div[3]/div/div/div/div[3]/div[2]/table/tbody/tr[{}]/td[9]').format(j));
                        vitderr = driver.find_element_by_xpath(('/html/body/div[7]/div[3]/div[6]/div[3]/div/div/div/div[3]/div[2]/table/tbody/tr[{}]/td[1]/div').format(j));
                    except:
                        pass
                    contrb = (('{},{},{}\n').format((current_game.get_attribute('href')), entrou_saiu.text, vitderr.text));
                    print(contrb)
                    path_folder = (('2017_2018/{}/{}.jg').format(files[e], jogador[0]))# e aqui
                    print(jogador[0])
                    with open(path_folder, 'a') as f:
                        f.write(contrb)
            else:
                pass

# Recenzja Gry

In [ ]:
def table_results():
    fora = driver.find_element_by_xpath('//div[@class="away"]//div[@class="team"]').text
    casa = driver.find_element_by_xpath('//div[@class="home"]//div[@class="team"]').text
    equipa_fora_golos = driver.find_elements_by_xpath('//div[@class="away"]//div[@class="scorers"]//span[@class="time"]')
    equipa_casa_golos = driver.find_elements_by_xpath('//div[@class="home"]//div[@class="scorers"]//span[@class="time"]')
    golls_fora : list = []
    golls_casa : list = []
    for g in equipa_fora_golos:
        gl : list = g.text.replace(' (g.p.)', '').replace(' (p.b.)', '').split(' ')
        e : int
        for e in range(len(gl)):
            golls_fora.append(eval(gl[e]))
        
    for g in equipa_casa_golos:
        gl : list = g.text.replace(' (g.p.)', '').replace(' (p.b.)', '').split(' ')
        e : int
        for e in range(len(gl)):
            golls_casa.append(eval(gl[e]))
                
    
    return casa, golls_casa, fora, golls_fora

In [ ]:
def goal_comparison(files, casa, goloj_casa, fora, goloj_fora):
    gm : int = 0
    gs : int = 0
    if files[10] == casa:
        print('casa')
        for g in range(len(goloj_casa)):
            if (goloj_casa[g] >= minutos_jogados[0]) and (goloj_casa[g] <= minutos_jogados[1]):
                gm += 1
        for g in range(len(goloj_fora)):
            if (goloj_fora[g] >= minutos_jogados[0]) and (goloj_fora[g] <= minutos_jogados[1]):
                gs += 1

    elif files[10] == fora:
        print('fora')
        for g in range(len(goloj_casa)):
            if (goloj_casa[g] >= minutos_jogados[0]) and (goloj_casa[g] <= minutos_jogados[1]):
                gs += 1
        for g in range(len(goloj_fora)):
            if (goloj_fora[g] >= minutos_jogados[0]) and (goloj_fora[g] <= minutos_jogados[1]):
                gm += 1
                
    return gm, gs

### L'Algorithme 

In [ ]:
n_jogadors : int = 0
path : str = '2017_2018/{}/jogadores.jfo'.format(files[10])
with open(path, 'r') as f:
    line : str
    for line in f:
        n_jogadors += 1
n_jogadors
for j in range(n_jogadors):
    files = os.listdir('2017_2018'); #files = Folders with team's names!
    jogadores = os.listdir('2017_2018/{}'.format(files[10]));
    path = ('2017_2018/{}/{}'.format(files[10],jogadores[j]))
    with open(path, 'r', encoding='utf-8') as f:
        l = 0
        for line in f:
            dados_jogo = line.split(',')
            driver.get(dados_jogo[0])
            casa, goloj_casa, fora, goloj_fora = table_results()
            print(casa, goloj_casa, fora, goloj_fora)
            minutos_jogados = 0
            if dados_jogo[1] == '-':
                minutos_jogados = [0, 95]
            elif dados_jogo[1][0] == '8':
                get_m = dados_jogo[1].replace('\'', '').split(' ')
                minutos_jogados = [0, int(get_m[1])]
            elif dados_jogo[1][0] == '7':
                get_m = dados_jogo[1].replace('\'', '').split(' ')
                minutos_jogados = [int(get_m[1]), 95]

            gm : int = 0
            gs : int = 0
            gm, gs = goal_comparison(files, casa, goloj_casa, fora, goloj_fora)
            tempo : int = (minutos_jogados[1] - minutos_jogados[0])
            path_sts : str = ('2017_2018/{}/stats/{}.sts'.format(files[10],jogadores[j][:-3]))
            t_write : str = '{},{},{}\n'.format(gm, tempo, gs)
            with open(path_sts , 'a', encoding='utf-8') as f:
                f.write(t_write)

# From here now, there'll only be ¡TESTS!